In [1]:
# loading libraries

library(repr)
library(tidyverse)
library(tidymodels)

options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [39]:
# reading data (from https://www.ultimatetennisstatistics.com/)

tennis_data <- read_csv("data/player_stats.csv")

# organizing/tidying data

colnames(tennis_data) = make.names(colnames(tennis_data))

tennis <- tennis_data |>
            # select(Age, Plays, Best.Rank, Backhand, Height, Favorite.Surface) |>  # Weight and Coach seem relevant, however there are too many missing values
            select(Best.Rank, Age, Height, Weight, Plays, Backhand, Favorite.Surface) |>
            mutate(across(everything(), function(col) {gsub(" .*", "", col)})) |>
            drop_na()
      
tennis

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Best.Rank,Age,Height,Weight,Plays,Backhand,Favorite.Surface
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
3,31,198,97,Right-handed,Two-handed,All-Rounder
1,33,185,85,Left-handed,Two-handed,Clay
3,22,198,86,Right-handed,Two-handed,Clay
⋮,⋮,⋮,⋮,⋮,⋮,⋮
13,24,193,85,Right-handed,Two-handed,Hard
3,29,196,98,Right-handed,Two-handed,Fast
4,26,185,82,Right-handed,One-handed,Clay
